In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Project1 import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
beginningValue = {}
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Grazioso Salvare database by Ryan Stork'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    dcc.RadioItems(
        id='RadioFilter-id',
        options=[
            {'label':'Water Rescue','value':'WR'},
            {'label':'Mountain Rescue','value':'MR'},
            {'label':'Disaster Rescue','value':'DR'},
            {'label':'Reset','value':'ALL'}
        ],
        value='ALL'),
    html.Hr(),   
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False,
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id', "data"),
     Output('graph-id', "children")],
    [Input('RadioFilter-id', 'value')]
)
def Animal_Radio_Button(animal_Radio):
    if animal_Radio == "WR":
        data={}
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                                                     "breed":{"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                                                     "outcome_type":{"$not": {"$in":["Euthanasia"]}},
                                                    "sex_upon_outcome":"Intact Female"}))
    elif animal_Radio == "MR":
        data={}
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                                                     "breed":{"$in": ["German Shepherd","Alaskan Malamute",
                                                                     "Old English Sheepdog","Siberian Husky","Rottweiler"]},
                                                     "outcome_type":{"$not": {"$in":["Euthanasia"]}},
                                                    "sex_upon_outcome":"Intact Male"}))
    elif animal_Radio == "DR":
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",
                                                     "age_upon_outcome_in_weeks":{"$gte":20,"$lte":300},
                                                     "breed":{"$in": ["Doberman Pinscher","German Shepherd",
                                                                     "Golden Retriever","Bloodhound","Rottweiler"]},
                                                     "outcome_type":{"$not": {"$in":["Euthanasia"]}},
                                                    "sex_upon_outcome":"Intact Male"
                                                     }))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    
    

    fig = px.pie(df,values=df.age_upon_outcome_in_weeks,names=df.breed,color_discrete_sequence=px.colors.sequential.RdBu,title='Breed Age')
    
    return df.to_dict('records'),dcc.Graph(figure=fig)








@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'active_cell'),
     Input('datatable-id', "derived_viewport_data")])
def update_map(ac, viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    if ac is not None:
        row = ac['row']
    else:
        row = 0   
    return [
        dl.Map(style = {'width': '1000px', 'height': '500px'}, center=[dff.loc[row,'location_lat'],dff.loc[row,'location_long']],zoom=10,children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[row,'location_lat'],dff.loc[row,'location_long']], children= [
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app